In [114]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.utils import resample
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, scale

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

df = pd.read_csv(
    r"/home/ahmed/Ai/Ml-algorithms-from-scratch-and-scikit-learn-in-python-/DataSets/UCI_Credit_Card.csv"
)
df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000.0,1,3,1,39,0,0,0,0,...,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29996,29997,150000.0,1,3,2,43,-1,-1,-1,-1,...,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29997,29998,30000.0,1,2,2,37,4,3,2,-1,...,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1
29998,29999,80000.0,1,3,1,41,1,-1,0,0,...,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1


In [115]:
df.rename({"default.payment.next.month": "default"}, axis="columns", inplace=True)
df.drop(columns=["ID"], axis="columns", inplace=True)

In [116]:
df.dtypes

LIMIT_BAL    float64
SEX            int64
EDUCATION      int64
MARRIAGE       int64
AGE            int64
PAY_0          int64
PAY_2          int64
PAY_3          int64
PAY_4          int64
PAY_5          int64
PAY_6          int64
BILL_AMT1    float64
BILL_AMT2    float64
BILL_AMT3    float64
BILL_AMT4    float64
BILL_AMT5    float64
BILL_AMT6    float64
PAY_AMT1     float64
PAY_AMT2     float64
PAY_AMT3     float64
PAY_AMT4     float64
PAY_AMT5     float64
PAY_AMT6     float64
default        int64
dtype: object

LIMIT_BAL amount of available creatid (float | int)

sex (1 male and 0 female)

eduction (1 graduate | 2 university | 3 high school | 4 others)

MARRIAGE (1 married | 2 single | 3 other)


In [117]:
df["SEX"].unique()

array([2, 1])

In [118]:
df["EDUCATION"].unique()

array([2, 1, 3, 5, 4, 6, 0])

#### is contains 0, 5 and 6, which it doesn't make sense


In [119]:
df["MARRIAGE"].unique()

array([1, 2, 3, 0])

#### is contains 0, which it doesn't make sense


In [120]:
len(df[(df["EDUCATION"] == 0) | (df["MARRIAGE"] == 0)])

68

In [121]:
df_with_no_missing = df[(df["EDUCATION"] != 0) & (df["MARRIAGE"] != 0)]
len(df_with_no_missing)

29932

In [122]:
df_default0 = resample(
    df_with_no_missing[df_with_no_missing["default"] == 0],
    replace=False,
    n_samples=2000,
    random_state=42,
)
df_default1 = resample(
    df_with_no_missing[df_with_no_missing["default"] == 1],
    replace=False,
    n_samples=2000,
    random_state=42,
)

df_resampled = pd.concat([df_default0, df_default1])
df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,20000.0,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,50000.0,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000.0,1,3,1,39,0,0,0,0,0,...,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29996,150000.0,1,3,2,43,-1,-1,-1,-1,0,...,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29997,30000.0,1,2,2,37,4,3,2,-1,0,...,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1
29998,80000.0,1,3,1,41,1,-1,0,0,0,...,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1


In [123]:
feature = df_resampled.drop(columns=["default"], axis="columns")
test = df_resampled["default"]
feature

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
641,130000.0,2,3,1,28,0,0,0,0,-2,...,50000.0,0.0,0.0,0.0,2500.0,1000.0,0.0,0.0,0.0,0.0
4678,170000.0,1,3,1,29,0,0,0,0,0,...,172307.0,35234.0,32869.0,33862.0,7200.0,7500.0,1200.0,1200.0,1500.0,1300.0
16004,180000.0,2,2,1,29,0,0,0,0,0,...,26310.0,26662.0,26166.0,26176.0,1800.0,1800.0,1500.0,1056.0,950.0,1000.0
22974,210000.0,2,2,2,32,-2,-2,-2,-2,-2,...,410.0,0.0,0.0,0.0,979.0,412.0,0.0,0.0,0.0,0.0
17535,190000.0,2,3,1,45,0,0,0,0,0,...,80548.0,81778.0,83082.0,84811.0,3300.0,3331.0,3359.0,2663.0,2751.0,3000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11697,240000.0,2,2,1,35,-1,-1,-1,-1,-1,...,1473.0,1473.0,1473.0,1473.0,1473.0,1473.0,1473.0,1473.0,1473.0,390.0
5597,30000.0,2,2,1,52,0,0,0,0,-1,...,21990.0,15525.0,25242.0,0.0,2000.0,2000.0,5000.0,25242.0,0.0,5000.0
25627,50000.0,2,2,1,30,0,0,0,0,0,...,27835.0,29070.0,30891.0,54741.0,1450.0,1800.0,2000.0,2600.0,0.0,2020.0
16231,480000.0,2,2,2,34,0,0,0,0,0,...,471639.0,375196.0,285319.0,209218.0,18231.0,20010.0,13013.0,11209.0,9010.0,162510.0


In [124]:
feature_encoded = pd.get_dummies(
    feature,
    columns=[
        "SEX",
        "EDUCATION",
        "MARRIAGE",
        "PAY_0",
        "PAY_2",
        "PAY_3",
        "PAY_4",
        "PAY_5",
        "PAY_6",
    ],
)
feature_encoded

,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,...,PAY_5_7,PAY_6_-2,PAY_6_-1,PAY_6_0,PAY_6_2,PAY_6_3,PAY_6_4,PAY_6_5,PAY_6_6,PAY_6_7
641,130000.0,28,100143.0,50456.0,50000.0,0.0,0.0,0.0,2500.0,1000.0,...,False,True,False,False,False,False,False,False,False,False
4678,170000.0,29,165027.0,168990.0,172307.0,35234.0,32869.0,33862.0,7200.0,7500.0,...,False,False,False,True,False,False,False,False,False,False
16004,180000.0,29,25781.0,26000.0,26310.0,26662.0,26166.0,26176.0,1800.0,1800.0,...,False,False,False,True,False,False,False,False,False,False
22974,210000.0,32,355.0,975.0,410.0,0.0,0.0,0.0,979.0,412.0,...,False,True,False,False,False,False,False,False,False,False
17535,190000.0,45,76433.0,78472.0,80548.0,81778.0,83082.0,84811.0,3300.0,3331.0,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11697,240000.0,35,1473.0,1473.0,1473.0,1473.0,1473.0,1473.0,1473.0,1473.0,...,False,False,True,False,False,False,False,False,False,False
5597,30000.0,52,20533.0,20819.0,21990.0,15525.0,25242.0,0.0,2000.0,2000.0,...,False,False,True,False,False,False,False,False,False,False
25627,50000.0,30,25422.0,26456.0,27835.0,29070.0,30891.0,54741.0,1450.0,1800.0,...,False,False,False,False,True,False,False,False,False,False
16231,480000.0,34,489305.0,479927.0,471639.0,375196.0,285319.0,209218.0,18231.0,20010.0,...,False,False,False,True,False,False,False,False,False,False


In [125]:
x_train, x_test, y_train, y_test = train_test_split(
    feature, test, test_size=0.2, random_state=42
)
x_train_scaled = scale(x_train)
x_test_scaled = scale(x_test)

### A blind SVM without parameters


In [126]:
svm = SVC(random_state=42)
svm.fit(x_train_scaled, y_train)
svm.score(x_test_scaled, y_test) * 100

69.375

## opitmized SVM using GridSearchCV


In [127]:
param_grid = {
    "C": [0, 0.5, 1, 2, 5, 100],
    "gamma": ["sacle", 1, 0.1, 0.01, 0.001, 0.0001, 0.00001],
    "kernel": ["rbf"],
}

opitmized_para = GridSearchCV(
    SVC(random_state=42), param_grid=param_grid, cv=5, scoring="accuracy"
)
opitmized_para.fit(x_train_scaled, y_train)

/home/ahmed/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:542: FitFailedWarning: 
60 fits failed out of a total of 210.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "/home/ahmed/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/ahmed/.local/lib/python3.10/site-packages/sklearn/base.py", line 1344, in wrapper
    estimator._validate_params()
  File "/home/ahmed/.local/lib/python3.10/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/home/ahmed/.local/lib

GridSearchCV(cv=5, estimator=SVC(random_state=42),
             param_grid={'C': [0, 0.5, 1, 2, 5, 100],
                         'gamma': ['sacle', 1, 0.1, 0.01, 0.001, 0.0001, 1e-05],
                         'kernel': ['rbf']},
             scoring='accuracy')

In [128]:
opitmized_para.best_params_

{'C': 5, 'gamma': 0.01, 'kernel': 'rbf'}

In [129]:
opitmized_svm = SVC(C=100, gamma=0.01, kernel="rbf")
opitmized_svm.fit(x_train_scaled, y_train)
opitmized_svm.score(x_test_scaled, y_test) * 100

70.625